In [50]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from re import sub

df = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)
df = df.iloc[:, :9]
df_ocr = pd.read_csv('20220209190321A17.csv', skiprows=1)

In [51]:
df_ocr.columns = ['no', 'id', 'fullname', 'party', 'vote']
df_ocr = df_ocr.astype(np.str_)

In [52]:
def similar(a, b):
    return SequenceMatcher(lambda x: x in ' ', a, b).ratio()

In [53]:
def fix_party(p, ps):
  if not p: return p

  for x in ps:
    sim = similar(p, str(x))
    if sim > .8:
      return x
  return p

df_ocr.party = df_ocr.party.apply(lambda x: fix_party(x.replace('พรรศ', 'พรรค').replace('พรวศ', 'พรรค').replace('พรรค', ''), df.party.unique()))

In [54]:
party_b = df_ocr.party.isin(df.party.dropna())

In [55]:
TITLE = ['ร้อยตํารวจตรี', 'พันเอก', 'ร้อยตํารวจเอก', 'พันตํารวจโท', 'พลตํารวจเอก', 'พันตํารวจเอก']
REPLACE = ('นนางสาว', 'นางสาว'), ('นษาง', 'นาง'), ('นบนาย', 'นาย'), ('บาย', 'นาย'), ('พสตํารวจ', 'พลตำรวจ'), ('พลตำรวจตรวี', 'พลตำรวจตรี')

def split_name(name, titles):
  name = sub('[!-~]', '', name)
  for o, n in REPLACE:
    if name.startswith(o):
      name = name.replace(o, n)

  name = sub('(ร้อย|พัน|พล)(ตํารวจ)?(ตรี|โท|เอก)', '', name).strip()

  for t in titles:
    if name.startswith(t):
      return name[len(t):]
  return name

df_ocr.fullname = df_ocr.fullname.apply(lambda x: split_name(x, df.title.unique().tolist()+TITLE))

In [56]:
def vote_encoder(vote: str):
  '''
  1 = เห็นด้วย,
  2 = ไม่เห็นด้วย,
  3 = งดออกเสียง,
  4 = ไม่ลงคะแนนเสียง, 
  5 = ไม่เข้าร่วมประชุม,
  \- = ไม่ใช่วาระการประชุม
  '''
  if vote == 'เห็นด้วย':
    return 1
  if vote == 'ไม่เห็นด้วย':
    return 2
  if vote == 'งดออกเสียง':
    return 3
  if vote == 'ไม่ลงคะแนนเสียง':
    return 4
  if vote == '-':
    return 5

  p = (similar(vote,'เห็นด้วย'), similar(vote,'ไม่เห็นด้วย'), similar(vote,'งดออกเสียง'), similar(vote, 'ไม่ลงคะแนนเสียง'))
  i = np.argmax(p)
  if p[i] < .8:
    return 5
  return i + 1

In [57]:
df_ocr.vote = df_ocr.vote.apply(vote_encoder)

In [58]:
name_b = df_ocr.fullname.apply(lambda x: x.split(' ')[0]).apply(lambda x: (x == df.name.dropna()).sum())

In [59]:
def last_name(fullname):
  sep_fn = fullname.split(' ', maxsplit=1)
  if (len(sep_fn) > 1):
    return sep_fn[-1].replace('\'', '')
  return fullname

last_name_b = df_ocr.fullname.apply(last_name).isin(df.lastname.dropna())

In [60]:
name_b.sum(),last_name_b.sum(),(name_b|last_name_b).sum(),party_b.sum()

(486, 312, 409, 471)

In [61]:
included_b = (name_b)|(last_name_b)|(party_b)
df_cleaned = df_ocr[included_b]
df_cleaned

,no,id,fullname,party,vote
6,.,383,สมบัติ ศรีสุรินทร์,เพื่อไทย,1
7,2,009,กตฤษฎา ตันเทอดทิตย์,เพื่อไทย,2
8,3,022,กิตตี๋ธัญญา วาจาดี,เพื่อไทย,2
9,4ง,023,คิตติศักดิ์ คณาสวัสดิ์,เพื่อไทย,2
10,5,035,ขนบายโกศสล ปทบมะ,เพื่อไทย,2
...,...,...,...,...,...
504,70,2365,บางสาวศีลับมพา เลิศนุวัฒน์,พลเมืองไทย,2
505,47],135,บุญญาพร นาตะธนภัทร,พลังชาติไทย,1
506,412,296,ระวิ มาศฉผูมาตล,พลังธรรมใหม่,1
507,ๆ73,179,นิคม บุญวิเศษ,พลังปวงชนไทย,5


In [62]:
match_row = np.full(df.name.shape, np.nan)

for i, g in df.groupby('party'):
  tmp = df_cleaned[df_cleaned['party'] == i]
  for j, r in tmp.iterrows():
    sim = g.apply(lambda x: similar(r.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
    if sim.max() > .6:
      if j not in match_row:
        match_row[g.iloc[sim.argmax()].name] = j

for i, row in df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)].iterrows():
  sim = df.apply(lambda x: similar(row.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
  if sim.max() > .7:
    match_row[sim.argmax()] = i

df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)]

,no,id,fullname,party,vote


In [63]:
df['ocr'] = match_row

In [64]:
merged_df = df.merge(df_ocr[['fullname', 'vote']], left_on='ocr', right_index=True)
merged_df

,id,title,name,lastname,party,is_mp,is_senator,is_cabinet,is_active,ocr,fullname,vote
21,22,นาย,กนก,ลิ้มตระกูล,เพื่อไทย,1.0,0.0,0.0,1,63.0,กนก ลิ้มตระกูล,5
22,23,ศาสตราจารย์ ดร.,กนก,วงษ์ตระหง่าน,ประชาธิปัตย์,1.0,0.0,0.0,1,365.0,กนก วงษ์ตระหง่าน,1
23,24,นาย,กมลศักดิ์,ลีวาเมาะ,ประชาชาติ,1.0,0.0,0.0,1,460.0,กมลศักดิ์ ลิวาเมาะ,3
25,26,นาง,กรณิศ,งามสุคนธ์รัตนา,พลังประชารัฐ,1.0,0.0,0.0,1,145.0,กรณีศ งามสุศนธ์รัตนา,1
26,27,นาย,กรวีร์,ปริศนานันทกุล,ภูมิใจไทย,1.0,0.0,0.0,1,249.0,กรวิร์ ปริศนานันทกุล,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1054,นาย,ชวน,ชูจันทร์,พลังประชารัฐ,1.0,0.0,0.0,1,220.0,ขวน ซูจันทร์,1
1054,1055,นาย,จักพันธ์,ปิยพรไพบูลย์,ประชาธิปัตย์,1.0,0.0,0.0,1,416.0,จักพันธ์ ปิยพรไพบูลย์,5
1055,1056,นางสาว,สุภาพร,กำเนิดผล,ประชาธิปัตย์,1.0,0.0,0.0,1,374.0,สุภาพร กําเนิดผล,1
1056,1057,นางสาว,สุเนตตา,แซ่โก๊ะ,รวมพลังประชาชาติไทย,1.0,0.0,0.0,1,483.0,สุเนตตา แซ่โก๊ะ,1


In [66]:
df_ch = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)

In [67]:
merged_df[(df_ch.loc[merged_df.index]['votelog.__82'].fillna(5).astype(np.int16)==merged_df.vote)==False]

,id,title,name,lastname,party,is_mp,is_senator,is_cabinet,is_active,ocr,fullname,vote
